In [ ]:
import pandas as pd
import os

In [ ]:
gwas_path = 'E:/downloads/gwas'
gwas_files = os.listdir(gwas_path)
print(gwas_files)

In [ ]:
# finngen gwas
gwas_files_finngen = [f for f in gwas_files if 'finngen' in f]

In [ ]:
df_asthma = pd.read_csv('E:/downloads/gwas/TAGC_Multiancestry_and_European-Ancestry_Meta-analyses_Results.tsv', sep='\t', low_memory=False)

In [ ]:
df_asthma.columns

In [ ]:
# change column names, rename to: CHR, BP, SNP, A1, A2, P, BETA, SE (other selected columns:  EAF, N)
df_asthma = df_asthma.rename(columns={
    'chr': 'CHR',
    'position': 'BP',
    'rsid': 'SNP',
    'alternate_allele': 'A1',
    'reference_allele': 'A2',
    'Multiancestry_beta_fix': 'BETA',
    'Multiancestry_se_fix': 'SE',
    'Multiancestry_pval_fix': 'P',
})
df_asthma = df_asthma[['CHR', 'BP', 'SNP', 'A1', 'A2', 'P', 'BETA', 'SE']]

In [ ]:
print(df_asthma.head())

In [ ]:
top_hits = df_asthma[df_asthma['P'] < 5e-8]

In [ ]:
# save to csv
df_asthma.to_csv('data/gwas_summary/disease/Asthma.txt.gz', index=False, sep=' ', compression='gzip', na_rep='NA')

In [ ]:
top_hits.to_csv('data/gwas_summary/disease/tophits/Asthma_top.csv', index=False, na_rep='NA')

In [ ]:
df_gwas = pd.read_csv('E:/downloads/gwas/finngen_R9_I9_CHD.gz', sep='\t', low_memory=False, compression='gzip')

In [ ]:
df_gwas.columns

In [ ]:
# change column names, rename to: CHR, BP, SNP, A1, A2, P, BETA, SE (other selected columns:  EAF, N)
df_gwas = df_gwas.rename(columns={
    '#chrom': 'CHR',
    'pos': 'BP',
    'rsids': 'SNP',
    'alt': 'A1',
    'ref': 'A2',
    'pval': 'P',
    'beta': 'BETA',
    'sebeta': 'SE',
    'af_alt': 'EAF',
})
df_gwas = df_gwas[['CHR', 'BP', 'SNP', 'A1', 'A2', 'P', 'BETA', 'SE', 'EAF']]

In [ ]:
# top hits
top_hits = df_gwas[df_gwas['P'] < 5e-8]

In [ ]:
# save to csv
df_gwas.to_csv('data/gwas_summary/disease/CHD.txt.gz', index=False, sep=' ', compression='gzip', na_rep='NA')
top_hits.to_csv('data/gwas_summary/disease/tophits/CHD_top.csv', index=False, na_rep='NA')

In [ ]:
for f in gwas_files_finngen:
    print('processing:', f)
    # ignore CHD because it is already processed
    if f == 'finngen_R9_I9_CHD.gz':
        continue
    df_gwas = pd.read_csv(f'{gwas_path}/{f}', sep='\t', low_memory=False, compression='gzip')
    df_gwas = df_gwas.rename(columns={
        '#chrom': 'CHR',
        'pos': 'BP',
        'rsids': 'SNP',
        'alt': 'A1',
        'ref': 'A2',
        'pval': 'P',
        'beta': 'BETA',
        'sebeta': 'SE',
        'af_alt': 'EAF',
    })
    df_gwas = df_gwas[['CHR', 'BP', 'SNP', 'A1', 'A2', 'P', 'BETA', 'SE', 'EAF']]
    top_hits = df_gwas[df_gwas['P'] < 5e-8]
    print('saving results, number of SNPs:', len(df_gwas), 'top hits:', len(top_hits))
    df_gwas.to_csv(f'data/gwas_summary/disease/{f}.txt.gz', index=False, sep=' ', compression='gzip', na_rep='NA')
    top_hits.to_csv(f'data/gwas_summary/disease/tophits/{f}_top.csv', index=False, na_rep='NA')

In [ ]:
df_gwas = pd.read_csv('data/gwas_summary/disease/GN.txt.gz', sep=' ', compression='gzip', low_memory=False)

In [ ]:
df_top = df_gwas[df_gwas['P'] < 5e-6]

In [ ]:
df_top.to_csv('data/gwas_summary/disease/tophits/GN_top.csv', index=False, na_rep='NA')